<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/Validation/SB2021/Docking_global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
import timeit
import os
import timeit
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import multiprocessing
from joblib import Parallel, delayed
condacolab.install()
!mamba install openbabel

time: 4.65 s (started: 2023-10-24 12:44:48 +00:00)


In [3]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! make -j2

time: 12.4 s (started: 2023-10-24 12:44:53 +00:00)


In [4]:
#@title Importing pyLiBELa {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!
time: 58 ms (started: 2023-10-24 12:45:05 +00:00)


In [14]:
#@title Auxiliary functions {display-mode: "form"}
def dock_mol (mol, Dock, Input, count):
  Lig2 = Mol2();
  if Lig2.parse_smiles(Input, mol, 'Lig'):
    Dock.run(REC, Lig2, RefLig, center, Input, Grids, count);

def dock_target(target, Input):
  target_folder = sb_folder + target
  %cd $target_folder
  !mkdir -p {Input.energy_optimizer}_SF{scoring_function}
  %cd {Input.energy_optimizer}_SF{scoring_function}
#  !rm -f *
  if (os.path.exists('../grids/McGrid_SF0') and not os.path.exists('McLiBELa_dock.mol2.gz')):

    REC = Mol2(Input, '../rec.mol2.gz');
    RefLig = Mol2(Input, '../lig.mol2.gz');
    Lig2 = Mol2();

    Writer = WRITER(Input);
    Coord = COORD_MC();
    HB = FindHB();
    Ene  = Energy2(Input);

    for i in range(len(REC.residue_pointer)-1):
      HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
    HB.find_ligandHB('../lig.mol2.gz', RefLig);
    print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

    insmiles = open('../lig2.smi', 'r');
    line=insmiles.readline().strip()
    line2 = line.split()
    ligand_smiles = line2[0]
    insmiles.close();

    if (Lig2.parse_smiles(Input, ligand_smiles, 'LIG')):

      Dock = Docker(Writer)
      center = Coord.compute_com(RefLig)

      print()
      start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
      print('Starting energy: %7.3f kcal/mol' % start_energy);
      print('Loading grids...')
      Grids = Grid(Input, Writer)
      Grids.load_grids_from_file();
      print('Grids loaded.')
      grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
      print('Grid original energy: %7.3f kcal/mol' % grid_energy);
      print('Grid error: %7.3f %%' % abs(100.*(start_energy-grid_energy)/start_energy));

      print('Starting docking calculation...')
      Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
      print('Docking calculation finished!')

time: 1.37 ms (started: 2023-10-24 13:09:01 +00:00)


In [6]:
#@title Getting SB2021 data from Google Drive {display-mode: "form"}

%%capture
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/Projects_Data/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()
targets = targets[:-2] #os últimos 2 são list e list.txt, só quero os pdb

time: 1.95 s (started: 2023-10-24 12:45:05 +00:00)


In [15]:
#@title Set Parameters for Grid Computation {display-mode: "form"}

Input = PARSER()

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = False;
Input.load_grid_from_file = True;
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 10000 #@param {type:"number"}

scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_prefix = '../grids/McGrid_SF0'
Input.grid_spacing = 0.4 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-4;
Input.dock_min_tol = 1E-4;
search_box = 8.0 #@param {type:"number"}
Input.timeout = 20 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "direct_only" #@param ["direct_only", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
    delta = 2.5 #@param {type:"number"}
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5 #@param {type:"number"}
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

#Input.use_overlay_cutoff = True # @param {type:"boolean"}
#Input.overlay_cutoff = 0.6 #@param {type:"number"}
Input.use_smiles = True;

time: 3.71 ms (started: 2023-10-24 13:09:04 +00:00)


In [ ]:

#@title Reading molecular files and Docking {display-mode: "form"}

ncores = multiprocessing.cpu_count()

print('Runing docking with %d threads.' % ncores)

subset = targets[:10]

parallel_docking = Parallel(backend='threading', n_jobs=1)(delayed(dock_target)(target, Input) for target in targets)

Runing docking with 2 threads.
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/121P
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/121P/direct_only_SF0
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/181L
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/181L/direct_only_SF0
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/182L
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/182L/direct_only_SF0
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/183L
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/183L/direct_only_SF0
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/184L
/content/drive/.shortcut-targets-by-id/1iX13EV9pFTj4bafAE1ugNFli9exvXFbC/pyLiBELa/SB/184L/direc